In [315]:
# Dependencies and Setup
from bs4 import BeautifulSoup as bs
from splinter import Browser
import json
import pandas as pd
import time
import datetime
import numpy as np
import os
import glob
import pprint

In [ ]:
# WINDOWS/PC - Import splinter and set the cromedriver path
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# MAC - Import splinter and set the cromedriver path
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#browser = Browser('chrome', **executable_path, headless=False)

# Load and Convert Baby Names and Year of Birth

In [ ]:
# load and read name files; add year column based on file name

fileList=glob.glob("*.txt")
#def addField(fileList):
for filename in fileList:
    #print(filename)
    df=pd.read_csv(filename, header=None)           
    df['yr']=[filename.rsplit("b",1)[1]]*df.shape[0]
    #print(df['yr'])
    df.to_csv(filename+".csv", index=None, header=None)

In [ ]:
# Add Column Headings, Concatenate files and merge to single csv file "Concatenated.csv"

outfile="C:Concatenated.csv"
fileList=glob.glob("*.csv")
dfList=[] # create an empty list
colnames=['Name', 'Gender', 'Count', 'Year']
for filename in fileList:
    #print(filename)
    df=pd.read_csv(filename, header=None)
    dfList.append(df)
concatDf=pd.concat(dfList, axis=0)
concatDf.columns=colnames

In [ ]:
# Sample output from the "Concatenated.csv" file
concatDf.head()

In [ ]:
# CODE for splitting .txt from Year field - Leaving only the year
newYr = concatDf["Year"].str.split(".",n=1, expand=True)
#newYr.head()
concatDf["Year"]=newYr[0]
concatDf["txt"]=newYr[1]
concatDf.drop(columns=['txt'], inplace=True)
concatDf.head()

In [ ]:
# Pass dataframe to a .csv file 
concatDf.to_csv(outfile, index=None)

# Load and Convert Movie Files

In [316]:
# Insert Movies Data from csv file
movies="Cleaned_tmdb_5000_movies.csv"
moviesDF=pd.read_csv(movies)
#moviesDF.head()
#print(moviesDF)

In [ ]:
def load_tmdb_movies(path):   #df = "Cleaned_tmdb_5000_movies.csv"
    df = pd.read_csv(path)
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.date())
    json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

In [363]:
# Insert Credits Data from csv file
credits="Cleaned_tmdb_5000_credits.csv"
creditsDF=pd.read_csv(credits)
creditsDF.drop(columns=['crew'], inplace=True)
creditsDF.head()

,movie_id,title,cast
0,5,Four Rooms,"[{""cast_id"": 42, ""character"": ""Ted the Bellhop..."
1,11,Star Wars,"[{""cast_id"": 3, ""character"": ""Luke Skywalker"",..."
2,12,Finding Nemo,"[{""cast_id"": 8, ""character"": ""Marlin (voice)"",..."
3,13,Forrest Gump,"[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""..."
4,14,American Beauty,"[{""cast_id"": 6, ""character"": ""Lester Burnham"",..."


In [364]:
# NEW CELL

# Extract the FIRST ROW of the "cast" column

counter = 0 
print(type(creditsDF))
character_df = pd.DataFrame(columns=['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'movie_id'])

for row in creditsDF.iterrows():
    counter = counter + 1
    print(counter)
    castListDF = row[1][2]
    print(row)
    #print(castListDF)
    #print(type(castListDF_5))

    # Convert 'cast' list of dictionaries to dataframe; drop 'credit_id' field
    ch_data = json.loads(castListDF)
    #print(type(ch_data))
    ch_df = pd.DataFrame(ch_data)
    ch_df['movie_id'] = row[1]['movie_id']
    character_df=character_df.append(ch_df)
        
    try:
        print(row)
    except JSONDecodeError as e:
        skip
        #print(e)
        #pass
        #print('Sorry. Information not available')
        #continue
    except Exception as e:
        skip
        #print(e)
        print('Sorry. Information not available')
        #print 'Row Data:' ,len(row),row
        continue
    #print(ch_df)
    #print(character_df)
    #if counter >= 342:
        #break

ch_df.drop(columns=['credit_id'], inplace=True)
character_df   

<class 'pandas.core.frame.DataFrame'>
1
(0, movie_id                                                    5
title                                              Four Rooms
cast        [{"cast_id": 42, "character": "Ted the Bellhop...
Name: 0, dtype: object)
(0, movie_id                                                    5
title                                              Four Rooms
cast        [{"cast_id": 42, "character": "Ted the Bellhop...
Name: 0, dtype: object)
2
(1, movie_id                                                   11
title                                               Star Wars
cast        [{"cast_id": 3, "character": "Luke Skywalker",...
Name: 1, dtype: object)


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 5315 (char 5314)

In [298]:
# Clean data for unique values in character names
pd.value_counts(df['character'])

# Export dataframe to "characters.csv" file
character_df.to_csv("characters.csv", index=True, header=True)

# LOADING TO MYSQL

In [ ]:
from sqlalchemy import create_engine
from config import remote_db_endpoint, remote_db_port
# CHANGE THE REMOTE_DATABASE SCHEMA INFO
from config import remote_gwsis_dbname, remote_gwsis_dbuser, remote_gwsis_dbpwd
import pymysql
import pandas as pd
pymysql.install_as_MySQLdb()

# CHANGE THIS endpoint: gwda-etl-project.c2supt4qfmve.us-east-2.rds.amazonaws.com

In [ ]:
engine = create_engine(f"mysql://{remote_gwsis_dbuser}:{remote_gwsis_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_gwsis_dbname}")
conn = engine.connect()

In [ ]:
names_df = pd.read_csv("Concatenated.csv")
names_load = names_df
names_load.to_sql(name='Names', if_exists='append', con=conn, chunksize=2000000, index=False)

In [ ]:
movies_df = pd.read_csv("Cleaned_tmdb_5000_movies.csv")
movies_load = movies_df
movies_load.to_sql(name='Movies', if_exists='append', con=conn, chunksize=2000000, index=False)

In [ ]:
credits_df = pd.read_csv("Cleaned_tmdb_5000_credits.csv")
credits_load = credits_df
credits_load.to_sql(name='Characters', if_exists='append', con=conn, chunksize=2000000, index=False)